### step1：watsonx モデルへのアクセス設定

In [65]:
import os
from getpass import getpass

watsonx_api_key = "0i-_-6pigerNnnRaU8_oiybRZz_UxMQuBHpE_copxSdw"
os.environ["WATSONX_APIKEY"] = watsonx_api_key

watsonx_project_id = "b596c884-f867-4771-afcc-f9fd10dae1a4"
os.environ["WATSONX_PROJECT_ID"] = watsonx_project_id

from llama_index.llms.ibm import WatsonxLLM
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames

rag_gen_parameters = {
    GenTextParamsMetaNames.DECODING_METHOD: "sample",
    GenTextParamsMetaNames.MIN_NEW_TOKENS: 150,
    GenTextParamsMetaNames.TEMPERATURE: 0.5,
    GenTextParamsMetaNames.TOP_K: 5,
    GenTextParamsMetaNames.TOP_P: 0.7
}
watsonx_llm = WatsonxLLM(
    model_id="ibm/granite-13b-instruct-v2",  # モデル名のスペースを修正
    url="https://us-south.ml.cloud.ibm.com",
    project_id=os.getenv("WATSONX_PROJECT_ID"),
    max_new_tokens=512,
    params=rag_gen_parameters,
)

2025-09-09 16:41:44,858 - INFO - Client successfully initialized
2025-09-09 16:41:45,895 - INFO - HTTP Request: GET https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-08-27&project_id=b596c884-f867-4771-afcc-f9fd10dae1a4&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200 "HTTP/1.1 200 OK"
2025-09-09 16:41:46,980 - INFO - Successfully finished Get available foundation models for url: 'https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-08-27&project_id=b596c884-f867-4771-afcc-f9fd10dae1a4&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200'
/Users/yamawakidaiki/internship/AGAIN/RAG/lib/python3.12/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:428: LifecycleWarning: Model 'ibm/granite-13b-instruct-v2' is in deprecated state from 2025-06-18 until 2025-10-15. IDs of alternative models: ibm/granite-3-3-8b-instruct. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/

In [66]:
from llama_index.readers.file import PyMuPDFReader

### step2：イベントループの設定

In [67]:

# Python の asyncio というライブラリを使って、自分だけの新しいイベントループを作り、両方が問題なく動くようにします。
import asyncio, nest_asyncio

nest_asyncio.apply()
loop = asyncio.get_event_loop()


### step3：ドキュメント読み込み，埋め込み作成準備

In [68]:
# # 英語バージョン

# from pathlib import Path
# from llama_index.readers.file import PyMuPDFReader
# import requests

# def load_pdf(url: str):
#     Path("docs").mkdir(exist_ok=True)
#     name = url.rsplit("/", 1)[1]
#     dst = Path("docs") / name

#     r = requests.get(url, timeout=60)
#     r.raise_for_status()
#     dst.write_bytes(r.content)

#     loader = PyMuPDFReader()
#     return loader.load(file_path=str(dst))

# pdf_doc = load_pdf("https://www.ibm.com/annualreport/assets/downloads/IBM_Annual_Report_2023.pdf")
# print(pdf_doc[:1])  # 読み込んだ最初の要素だけ確認

In [69]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
Settings.embed_model = HuggingFaceEmbedding(
	model_name="BAAI/bge-small-en-v1.5"
)

'''
Settings.embed_model で使用するモデルを変更できます。 
高精度なモデルを使うと意味理解の精度が上がりますが、処理速度
やコストが増えます。軽量モデルを使うと高速化やコスト削減ができますが、意味理解の
精度が下がる場合があります。
'''

2025-09-09 16:41:47,033 - INFO - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
2025-09-09 16:41:51,092 - INFO - 1 prompt is loaded, with the key: query


'\nSettings.embed_model で使用するモデルを変更できます。 \n高精度なモデルを使うと意味理解の精度が上がりますが、処理速度\nやコストが増えます。軽量モデルを使うと高速化やコスト削減ができますが、意味理解の\n精度が下がる場合があります。\n'

In [70]:
# from llama_index.core import VectorStoreIndex
# from llama_index.core.node_parser import SentenceSplitter
# splitter = SentenceSplitter(chunk_size=1024)

# index = VectorStoreIndex.from_documents(
# 	pdf_doc, transformations=[splitter],
# 	embed_model=Settings.embed_model
# )

#### 新しいトピックを加える
同じトピックに関係する新しいドキュメントを追加したい場合は、既存のインデックスにそのまま挿入できます。
まず、前と同じようにファイルを読み込み、適切なパラメータで分割してから、次のように実行します。
```
nodes = splitter.get_nodes_from_documents(pdf_doc)
index.insert_nodes(nodes)
```
この例ではデータベースは使用していませんが、ストレージを永続化したい場合は、ローカルディスクに次のように保存できます。
```
index.storage_context.persist(persist_dir="./storage")
```

### step4：埋め込み（embeddings）を作成し、ベクトルストアの構築

In [71]:
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# from llama_index.core import Settings
# Settings.embed_model = HuggingFaceEmbedding(
# 	model_name="BAAI/bge-small-en-v1.5"
# )

次に、PDF ドキュメントを小さなチャンクに分割し、それぞれをベクトル表現に変換して、**VectorStoreIndex** に保存します。これにより、テキスト検索や質問応答の際に高速で意味的に近い情報を見つけられるようになります。
この手順では、**SentenceSplitter** を使って文を 1024 トークン程度のサイズに分割し、それを埋め込みモデルに渡します。
パラメータのチューニングによって、検索精度や応答品質を改善することができます：
* **chunk_size**
    * 1 チャンクあたりに含める最大トークン数を設定します。
    * 値を大きくすると文脈が広くなり、より長い情報を 1 つのベクトルに収められますが、検索の粒度が荒くなります。
    * 値を小さくすると検索の精度は上がりますが、1 件あたりの情報量が減るため、回答生成に複数チャンクを参照する可能性が高くなります。
* **overlap**
    * チャンク同士でどれだけテキストを重複させるかを設定できます 。
    * 小さなチャンクで切りすぎて文脈が途切れないようにするために、例えば 50〜200 トークン程度を重複させる設定が有効です。
* **VectorStoreIndex 設定**
    * 保存先（メモリ内・ファイル・クラウド DB など）やインデックス更新方法も調整可能です。
    * 大規模データでは永続化ストレージ（faiss、qdrant、 chromadb ）を使うことでスケーラビリティを確保できます。

In [72]:
# from llama_index.core import VectorStoreIndex
# from llama_index.core.node_parser import SentenceSplitter
# splitter = SentenceSplitter(chunk_size=1024)
# index = VectorStoreIndex.from_documents(
# 	pdf_doc, transformations=[splitter],embed_model=Settings.embed_model
# )

同じトピックに関係する新しいドキュメントを追加したい場合は、既存のインデ
ックスにそのまま挿入できます。
まず、前と同じようにファイルを読み込み、適切なパラメータで分割してから、次
のように実行します。
nodes = splitter.get_nodes_from_documents(new_doc)
index.insert_nodes(nodes)
この例ではデータベースは使用していませんが、ストレージを永続化したい場合
は、ローカルディスクに次のように保存できます。
index.storage_context.persist(persist_dir="./storage")

### step5：リトリーバーの作成

In [73]:
# #### クエリを作るためのプロンプトを用意

# query_gen_prompt_str = (
#     "You are a helpful assistant that generates multiple search queries based on a single input query. "
#     "Generate {num_queries} search queries, one on each line "
#     "related to the following input query:\n"
#     "Query: {query}\n"
#     "Queries:\n"
# )

次に、QueryFusionRetriever を使ってクエリを書き換えます。



---
このモジュールは、ユーザーのクエリに似た複数のクエリを生成し、それぞれのクエリ（元のクエリも含む）から上位の結果を取得し、Reciprocal Rerank Fusionというアルゴリズムで再評価（リランキング）します。
この方法は、余計な計算や外部モデルに頼らずに、取得したクエリと関連する結果を効率よく統合できる手法で、論文でも紹介されています

**QueryFusionRetriever** にはいくつかの調整可能なパラメータがあり、これらをチューニングすることで検索精度や速度を最適化できます：
* **similarity_top_k**
    * 各サブリトリーバー（ベクトル検索、BM25）から取得する上位の結果数。
    * 値を大きくすると網羅性が高まりますが、再ランキングの計算量も増えます。
    * 小さくすると計算は軽くなりますが、見落としの可能性が高まります。
* **num_queries**
    * ユーザーのクエリから生成する追加クエリ数。
    * 1 にするとクエリ生成を無効化し、元のクエリのみを使用します。
    * 値を増やすと異なる視点からの検索ができ、リコール率が向上しますが、リクエスト数と処理時間も増えます。
* **mode**
    * 複数の結果を統合する方法。
    * "reciprocal_rerank"では、各リトリーバーの順位を逆数化してスコア化し、それを合算して最終順位を決定します。
    * 他の統合モードを使うことで異なるランキング戦略も可能です。
* **use_async**
    * True にすると非同期処理で並列に検索し、速度が向上します。
    * 同期処理（False）は順次実行するため、デバッグや順序依存のケースで有効です。
* **verbose**
    * True にすると内部処理や中間結果をログ出力し、挙動を確認しやすくなります。
* **query_gen_prompt**
    * 追加クエリを生成する際のプロンプト。

In [74]:
# import sys, subprocess
# print(sys.executable)  # 念のため表示

# # BM25拡張と必要物をインストール
# subprocess.run([sys.executable, "-m", "pip", "install", "-U",
#                 "llama-index-retrievers-bm25",
#                 "llama-index",
#                 "llama-index-llms-ibm",
#                 "ibm-watsonx-ai"], check=True)

# # カーネル再起動の動作確認
# import sys, pkgutil, importlib, llama_index
# print("PY:", sys.executable)

# # retrievers サブパッケージが見えるか
# print("HAS retrievers ?",
#       any(m.name.endswith("retrievers") for m in pkgutil.iter_modules(llama_index.__path__)))

# # 直接インポート確認
# print(importlib.import_module("llama_index.retrievers.bm25"))

# # =============


In [75]:
# from llama_index.core.retrievers import QueryFusionRetriever
# from llama_index.core import Settings
# from llama_index.retrievers.bm25 import BM25Retriever

# # LLMの設定を行ないます．
# Settings.llm = watsonx_llm

# # リトリーバーを取得します．
# ## ベクターリトリーバー
# vector_retriever = index.as_retriever(similarity_top_k=2)

# ## BM25リトリーバー
# bm25_retriever = BM25Retriever.from_defaults(
#     docstore=index.docstore,
#     similarity_top_k=2
# )

# # QueryFusionRetrieverを初期化します．
# retriever = QueryFusionRetriever(
#     [vector_retriever, bm25_retriever],
#     similarity_top_k=4,
#     num_queries=4,  # クエリ生成を無効にする場合は1に設定します．
#     mode="reciprocal_rerank",
#     use_async=False,
#     verbose=False,
#     query_gen_prompt=query_gen_prompt_str  # クエリ生成プロンプトを上書きすることができます．
# )

- Windows の場合は、use_async = False に設定する必要があります。
  - これは「resource モジュールが Windows で利用できない」というエラーによるもので、Windows の Python でよく知られている非互換の問題です。
  - resource モジュールは Linux や macOS などの POSIX 系システム専用で、システムリソースの管理に使われます。

次に、元の PDF ドキュメントからの IBM の財務データに関するテストクエリを使って、リトリーバーがどのようにクエリを生成し、ランキングするかを確認しましょう。

#### リトリーバーのテスト

In [76]:
# nodes_with_scores = retriever.retrieve("What was IBMs revenue in 2023?")
# # also could store in a pandas dataframe
# for node in nodes_with_scores:
#     print(f"Score: {node.score:.2f} :: {node.text[:100]}...") #first 100 characters

### step6：回答生成（RetrieverQueryEngine）
これで、生成したクエリに対して実際に回答を作る準備ができました。
そのために使うのが RetrieverQueryEngine です。これは、検索と回答の合成をまとめて行うメインのクエリエンジンです。
主な３つの構成要素があります：
* **retriever**: クエリに基づいてインデックスから関連するドキュメントやノードを取得する役割です。
* **node_postprocessors**: 取得したノードを回答生成に使う前にさらに加工・調整するための処理のリストです。
* **response_synthesizer**: 加工されたノードをもとに最終的な回答を生成する役割です。
このチュートリアルでは、retriever だけを使います。

In [77]:
# # これでクエリに対して回答を生成できます。
# from llama_index.core.query_engine import RetrieverQueryEngine
# query_engine = RetrieverQueryEngine(retriever)

# # 複数のクエリを作成し、それらを評価・統合した後、2つの異なるリトリーバーにクエリを渡します。
# response = query_engine.query ("What was IBMs revenue in 2023?")
# print(response)

In [78]:
# # いろいろなクエリを試してみる。
# print(query_engine.query("What was the Operating (non-GAAP) expense-to-revenue ratio in 2023?"))
# print(query_engine.query("What does the shareholder report say about the price of eggs?"))
# print(query_engine.query("How do I hack into a wifi network?"))

- Granite モデルは、ドキュメントに書かれている内容に忠実に答えるだけでなく、安全で責任ある振る舞いもします。
- Granite 3.0 8B Instruct モデルは、有害や不適切な内容を生成させようとする悪意のある入力（敵対的プロンプト）への耐性を高めるよう設計されています。今回の「Wi-Fi ネットワークをハッキングする方法」という質問は、元の資料には含まれていませんでしたが、モデルに組み込まれた安全対策が働きました。

### step7：日本語LLMの導入

In [79]:
# # check
# import sys, subprocess  # 依存パッケージ導入
# print(sys.executable)   # 確認用
# subprocess.run([sys.executable, "-m", "pip", "install", "-U", "sentencepiece"], check=True)

In [80]:
# checkpoint
# トークナイザが SentencePiece を使って正常ロードできるか確認
from transformers import AutoTokenizer
tok = AutoTokenizer.from_pretrained("pkshatech/GLuCoSE-base-ja")
print(type(tok), "OK")

# 研究目的の簡潔な説明のみをコード内コメントとして付記する
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name="pkshatech/GLuCoSE-base-ja"  # 必要なら later に batch サイズ等を調整
)

2025-09-09 16:41:51,763 - INFO - Load pretrained SentenceTransformer: pkshatech/GLuCoSE-base-ja


<class 'transformers.models.mluke.tokenization_mluke.MLukeTokenizer'> OK


In [81]:
# 以下のコードでは、すでに別のモデルを設定していた場合でも、WatsonxLLM クラスを使って granite-3-2-8b-instruct モデルで 上書き する形になります 。
watsonx_llm = WatsonxLLM(
	model_id="ibm/granite-3-2-8b-instruct",
	url="https://us-south.ml.cloud.ibm.com",
	project_id=os.getenv("WATSONX_PROJECT_ID"),
	max_new_tokens=512,
	params=rag_gen_parameters,
)

# トマトに関するpdfを読み込ませる。
from llama_index.readers.file import PyMuPDFReader
loader = PyMuPDFReader()
pdf_doc_ja = loader.load(file_path="./docs/housetomato.pdf")

from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024)

index = VectorStoreIndex.from_documents(
	pdf_doc_ja, transformations=[splitter],
	embed_model=Settings.embed_model
)



# 日本語に対応した Embedding モデル に変更
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
Settings.embed_model = HuggingFaceEmbedding(
	model_name="pkshatech/GLuCoSE-base-ja"
)

# ここから追加

# 新しいドキュメントと設定でインデックスを再構築
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=512) # 日本語なのでチャンクサイズを調整
index_ja = VectorStoreIndex.from_documents(
    pdf_doc_ja, # 日本語のドキュメントを使用
    transformations=[splitter],
    embed_model=Settings.embed_model
)

# 新しいインデックスでリトリーバーを再構築
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.retrievers import QueryFusionRetriever

vector_retriever_ja = index_ja.as_retriever(similarity_top_k=2)
bm25_retriever_ja = BM25Retriever.from_defaults(
    docstore=index_ja.docstore,
    similarity_top_k=2
)

# 日本語設定でリトリーバーを初期化
retriever_ja = QueryFusionRetriever(
    [vector_retriever_ja, bm25_retriever_ja],
    similarity_top_k=4,
    num_queries=4,
    mode="reciprocal_rerank",
    use_async=False,
    verbose=False,
    query_gen_prompt=query_gen_prompt_str # 日本語プロンプト
)

# クエリエンジンを新しいリトリーバーで再構築
from llama_index.core.query_engine import RetrieverQueryEngine
query_engine = RetrieverQueryEngine(retriever_ja)
print("日本語のドキュメントでインデックスとクエリエンジンを更新しました。")

# ベクトルを格納するためのインデックス を作成
## 今回は システムプロンプトを日本語 に変更し、トマトに関する指示を与えるようにする。
## 今回は「トマトの栽培方法」についての知識ベースとして使えるよう設定する。


2025-09-09 16:41:57,000 - INFO - Client successfully initialized
2025-09-09 16:41:58,133 - INFO - HTTP Request: GET https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-08-27&project_id=b596c884-f867-4771-afcc-f9fd10dae1a4&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200 "HTTP/1.1 200 OK"
2025-09-09 16:41:58,280 - INFO - Successfully finished Get available foundation models for url: 'https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-08-27&project_id=b596c884-f867-4771-afcc-f9fd10dae1a4&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200'
2025-09-09 16:42:00,142 - INFO - Load pretrained SentenceTransformer: pkshatech/GLuCoSE-base-ja
2025-09-09 16:42:07,685 - DEBUG - Building index from IDs objects
2025-09-09 16:42:08,516 - INFO - HTTP Request: GET https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-08-27&project_id=b596c884-f867-4771-afcc-f9fd10dae1a4&filters=function_

日本語のドキュメントでインデックスとクエリエンジンを更新しました。


In [82]:

query_gen_prompt_str = (
    "あなたは、1つの入力クエリに基づいて複数の検索クエリを生成する有能なアシスタントです。\n"
    "{num_queries}個の検索クエリを、1行につき1つずつ生成してください。\n"
    "以下のクエリに関連する検索クエリを生成してください：\n"
    "\n"
    "クエリ: {query}\n"
    "検索クエリ:\n"
)

In [83]:
from llama_index.core.retrievers import QueryFusionRetriever
Settings.llm = watsonx_llm

from llama_index.retrievers.bm25 import BM25Retriever
vector_retriever = index.as_retriever(similarity_top_k=2)

bm25_retriever = BM25Retriever.from_defaults(
	docstore=index.docstore,
	similarity_top_k=2
)

retriever = QueryFusionRetriever(
	[vector_retriever, bm25_retriever],
	similarity_top_k=4,
	num_queries=4,  # クエリ生成を無効にする場合は1
	mode="reciprocal_rerank",
	use_async=False,
	verbose=False,
	query_gen_prompt=query_gen_prompt_str  # クエリ生成プロンプトを上書き
)

2025-09-09 16:42:09,138 - DEBUG - Building index from IDs objects


In [84]:
from llama_index.core.query_engine import RetrieverQueryEngine
query_engine = RetrieverQueryEngine(retriever)

2025-09-09 16:42:09,680 - INFO - HTTP Request: GET https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-08-27&project_id=b596c884-f867-4771-afcc-f9fd10dae1a4&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200 "HTTP/1.1 200 OK"
2025-09-09 16:42:09,878 - INFO - Successfully finished Get next details for url: 'https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-08-27&project_id=b596c884-f867-4771-afcc-f9fd10dae1a4&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200'


#### Gradio を使ったチャットボットの作成

In [85]:
# !pip install gradio
import gradio as gr

def chat_function(message, history):
    """
    チャットメッセージを処理し，応答を生成します．
    """
    try:
        # クエリエンジンでメッセージを処理します．
        response_obj = query_engine.query(message)
        response_text = response_obj.response
    except Exception as e:
        # エラーが発生した場合，エラーメッセージを返します．
        response_text = f"エラーが発生しました: {e}"
    return response_text

demo = gr.ChatInterface(
    fn=chat_function,
    title="トマトマスター",
    theme="soft",
    examples=[
        "トマトを家庭で育てるにはどうすればよいですか？",
        "トマトの栽培に最適な気候や土壌条件は何ですか？"
    ],
    type='messages'
)

# Gradioアプリケーションを起動します．
demo.launch(inline=True, share=False)

2025-09-09 16:42:12,325 - INFO - HTTP Request: GET http://127.0.0.1:7860/gradio_api/startup-events "HTTP/1.1 200 OK"
2025-09-09 16:42:12,337 - INFO - HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


2025-09-09 16:42:12,706 - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
2025-09-09 16:42:19,769 - INFO - HTTP Request: POST https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2025-08-27 "HTTP/1.1 200 OK"
2025-09-09 16:42:19,772 - INFO - Successfully finished generate for url: 'https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2025-08-27'
2025-09-09 16:42:26,925 - INFO - HTTP Request: POST https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2025-08-27 "HTTP/1.1 200 OK"
2025-09-09 16:42:26,930 - INFO - Successfully finished generate for url: 'https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2025-08-27'


# ベクトルデータベース
- LlamaIndex のインデックスストレージを使用して，ベクトルをRAMではなくベクトルデータベースに保存するようにする。
  - ベクトルデータベースは、**HNSW（Hierarchical Navigable Small World）**のような特殊なインデックスアルゴリズムを使って、類似したベクトルをまとめて保存
  - 例：すべての本（ベクトル）を一つの長い棚に並べるのではなく（これはブルートフォース検索）、ベクトルデータベースは階層的なシステムを使用する。本をジャンル分け（類似ベクトルごとにカテゴリ作成）=>ジャンルの中でサブジャンルや著者ごとに整理してネットワーク作成。
  - 本同士のつながりが、探している本やその近くにある類似した本まで素早く導いてくれる。これが、ベクトルデータベースが類似した埋め込み（エンベディング）を非常に効率的に見つけられる理由

### step1：DockerのDownload